# DDI InfluxDB Timeseries

### Imports

In [ ]:
# Imports
import os 
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
import time
from influxdb_client import Point
from libs.fetch_data import fetch_data

### Load of env Variables

In [ ]:
# Constants
load_dotenv(dotenv_path=".env")

# URLs und Token laden wir weiter aus der .env (das funktioniert ja)
URL   = os.getenv("INFLUX_URL")
ORG   = os.getenv("INFLUX_ORG", "basel-lab")
TOKEN = os.getenv("INFLUX_TOKEN")
PARKING_URL = os.getenv("PARKING_URL")
RAIN_URL = os.getenv("RAIN_URL")
ALRDY_INSTALLED = os.getenv("ALRDY_INSTALLED", 'False').lower() in ('true', '1', 't') 

# WICHTIG: Hier keine getenv() Funktion nutzen, sondern den Namen direkt zuweisen!
BUCKET = "parking_data_final"

print(f"Der Bucket ist gesetzt auf: '{BUCKET}'")

### Fetch of Data

In [ ]:
if not ALRDY_INSTALLED:
    df_parking, df_rain = fetch_data(PARKING_URL, RAIN_URL)
    df_parking.to_csv("../data/parking.csv")
    df_rain.to_csv("../data/rain.csv")

### Data analysis for import in InfluxDB

In [ ]:

df_parking = pd.read_csv("../data/parking.csv", sep=",")
df_rain = pd.read_csv("../data/rain.csv", sep=",")

ts = pd.to_datetime(df_parking['published'], errors="coerce", utc=True)
df_parking['time'] = ts


df_rain['date'] = pd.to_datetime(df_rain['date'], errors="coerce")
df_rain['time'] = (
    df_rain['date']
    + pd.to_timedelta(12, unit="h")
).dt.tz_localize("Europe/Zurich", nonexistent="shift_forward", ambiguous="NaT"
).dt.tz_convert("UTC")

In [ ]:
df_rain.head(10)

In [ ]:
df_parking.head(10)

### Make Connection

In [ ]:
# Connection 
assert TOKEN, "Bitte Token in .env file hinzufügen"
client = InfluxDBClient(url=URL, token=TOKEN, org=ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()
health = client.health()
print("InfluxDB health:", health.status, "-", health.message)


### Daten in das Model schreiben

In [ ]:
# ==========================================
# KONFIGURATION
# ==========================================
# Wir reduzieren die Menge pro Paket drastisch, um Timeouts zu verhindern
BATCH_SIZE = 1000  
PAUSE_SEC = 0.1    # Kurze Pause zwischen den Batches

# ==========================================
# 1. PARKDATEN
# ==========================================
print("Erstelle 'Points' für Parkdaten...")

points_parking = []
for index, row in df_parking.iterrows():
    if pd.notnull(row['time']): 
        p = Point("parking") \
            .tag("parking_id", str(row['id'])) \
            .tag("parking_name", str(row['title'])) \
            .field("free", float(row['free'])) \
            .field("total", float(row['total'])) \
            .time(row['time'])
        points_parking.append(p)

print(f"-> {len(points_parking)} Park-Punkte bereit. Starte Upload in {BATCH_SIZE}er Schritten...")

# Sicherer Batch-Loop
for i in range(0, len(points_parking), BATCH_SIZE):
    batch = points_parking[i : i + BATCH_SIZE]
    write_api.write(bucket=BUCKET, org=ORG, record=batch)
    print(f"   Park-Batch {i} bis {i + len(batch)} geschrieben.")
    time.sleep(PAUSE_SEC) # Kurze Atempause für die DB

print("Parkdaten fertig hochgeladen.\n")


# ==========================================
# 2. WETTERDATEN
# ==========================================
print("Erstelle 'Points' für Wetterdaten...")

points_rain = []
for index, row in df_rain.iterrows():
    if pd.notnull(row['time']):
        regen = float(row['rre150d0']) if pd.notnull(row['rre150d0']) else 0.0
        temp = float(row['tre200d0']) if pd.notnull(row['tre200d0']) else 0.0
        
        p = Point("weather") \
            .tag("location", "Basel") \
            .field("precipitation", regen) \
            .field("temperature", temp) \
            .time(row['time'])
        points_rain.append(p)

print(f"-> {len(points_rain)} Wetter-Punkte bereit. Starte Upload in {BATCH_SIZE}er Schritten...")

# Sicherer Batch-Loop
for i in range(0, len(points_rain), BATCH_SIZE):
    batch = points_rain[i : i + BATCH_SIZE]
    write_api.write(bucket=BUCKET, org=ORG, record=batch)
    print(f"   Wetter-Batch {i} bis {i + len(batch)} geschrieben.")
    time.sleep(PAUSE_SEC)

print("\n------------------------------------------------")
print("FERTIG! Alle Daten wurden erfolgreich importiert.")
print("------------------------------------------------")